In [1]:
#for questions: karenmei@ucsd.edu

#Goal: To evaluate the custom hierarchy: #Metric 1
#	   Metric #1: How well does the model capture novel synapse proteins? (some of which have been recently uncovered by our collaborators using AP/MS/MS)

#	   code for customizing ontologies: DDOT: https://github.com/michaelkyu/ddot/blob/master/examples/Tutorial.ipynb
import numpy as np
from igraph import *
import pandas as pd
import sys

import sys
sys.path.append('C:\\Users\\Anubhav\\Documents\\GitHub\\ddot')

import ddot
from ddot import Ontology
import matplotlib
#matplotlib.use("TKAgg")
#print(matplotlib.get_backend())
from matplotlib import pyplot as plt
import networkx as nx
import csv
import scipy.stats as ss
from scipy.stats import hypergeom
from statsmodels.sandbox.stats.multicomp import multipletests

from collections import defaultdict
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import networkx as nx 

import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
def jaccard(a, b):
	s1=set(a)
	s2=set(b)
	return len(s1.intersection(s2)) / len(s1.union(s2))

def metric_1(ont_file, test_gene_list):
	ont1=Ontology.from_table(ont_file)
	ont1_genes=ont1.genes
	test_recovery=jaccard(ont1_genes, test_gene_list)
	print ('recovery of test genes:', test_recovery)
	return test_recovery

In [3]:
def ontology_recursion(node, ontology, dic): 
    
    # Get children of node 
    children_df = ontology[ontology.Parent == node]

    # Recursion portion 
    for idx, child_sr in children_df.iterrows(): 

        if child_sr.EdgeType == 'Gene-Term':
            dic[node].add(child_sr.Child)
            
        elif child_sr.EdgeType == 'Child-Parent': 
            
            child_genes = ontology_recursion(child_sr.Child, ontology, dic)
            dic[node].update(child_genes)
        
    return(dic[node])

# Start at root 
global dic
root = 'GO:0045202'
synapse_ont = pd.read_table("synapse.txt")
dic = defaultdict(set)
x = ontology_recursion(root, synapse_ont, dic)